In [20]:
import pandas as pd
import numpy as np
import time
import os
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [26]:
df = pd.read_csv(r"C:\Users\P PAVAN REDDY\Documents\My Files\Projects\Recommendation System - Movies\movies.csv")

In [27]:
movies = df.copy()

In [28]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6147 entries, 0 to 6146
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   title                         6147 non-null   object
 1   release_year                  6147 non-null   int64 
 2   imdb_id                       6147 non-null   object
 3   poster                        6147 non-null   object
 4   tmdb_id                       6147 non-null   int64 
 5   cast                          6147 non-null   object
 6   director                      6147 non-null   object
 7   genres                        6147 non-null   object
 8   production_companies          6147 non-null   object
 9   cast_display                  6147 non-null   object
 10  director_display              6147 non-null   object
 11  genres_display                6147 non-null   object
 12  production_companies_display  6147 non-null   object
 13  title_search      

In [39]:
movies['title'].apply(lambda s : s.translate({ord(i): '' for i in " +:-,;$!|&*%"}))

0                         29thStreet
1                    TheAddamsFamily
2               AllIWantforChristmas
3                  AmericanKickboxer
4       AnAmericanTailFievelGoesWest
                    ...             
6142                        IronMan3
6143             AvatarTheWayofWater
6144         StarWarsTheForceAwakens
6145             StarWarsTheLastJedi
6146                        Furious7
Name: title, Length: 6147, dtype: object

In [44]:
def remove_special_characters(string):
    # Define the set of special characters to remove
    special_characters = " !@#$%^&*()_+=-{}[]|\:;\"'<>?,./"

    # Iterate over each character in the string and filter out the special characters
    cleaned_string = ''.join(char for char in string if char not in special_characters)

    return cleaned_string.lower()

# Example usage
string_with_special_chars = "H@e*llo# W&or%ld!"
cleaned_string = remove_special_characters(string_with_special_chars)
print(cleaned_string)  # Output: Hello World


helloworld


In [46]:
movies['title_search'] 

0                         29thstreet
1                    theaddamsfamily
2               alliwantforchristmas
3                  americankickboxer
4       anamericantailfievelgoeswest
                    ...             
6142                        ironman3
6143             avatarthewayofwater
6144         starwarstheforceawakens
6145             starwarsthelastjedi
6146                        furious7
Name: title_search, Length: 6147, dtype: object

In [47]:
movies['title_search'] = movies['title'].apply(remove_special_characters)

In [49]:
movies.to_csv('movies.csv', index=False)

creating tags column by concatinating 'cast', 'director', 'genres', 'production_companies'

In [24]:
movies['tags'] = (movies['cast'] + " " + movies['director'] + " " 
                       + movies['genres'] + movies['production_companies']
                      )

In [13]:
movies.head()

,title,release_year,imdb_id,poster,tmdb_id,cast,director,genres,production_companies,tags
0,29th Street,1991,tt0101252,https://m.media-amazon.com/images/M/MV5BMWY2MT...,43267,AnthonyLaPaglia DannyAiello LainieKazan FrankP...,GeorgeGallo,Comedy,20thCenturyFox,AnthonyLaPaglia DannyAiello LainieKazan FrankP...
1,The Addams Family,1991,tt0101272,https://m.media-amazon.com/images/M/MV5BODc1Nm...,2907,RaúlJuliá AnjelicaHuston ChristopherLloyd Chri...,BarrySonnenfeld,Comedy Fantasy,ScottRudinProductions OrionPictures Paramount,RaúlJuliá AnjelicaHuston ChristopherLloyd Chri...
2,All I Want for Christmas,1991,tt0101301,https://m.media-amazon.com/images/M/MV5BMTk3Mj...,13644,HarleyJaneKozak JameySheridan ThoraBirch Ethan...,RobLieberman,Family Drama Comedy,Paramount,HarleyJaneKozak JameySheridan ThoraBirch Ethan...
3,American Kickboxer,1991,tt0101325,https://m.media-amazon.com/images/M/MV5BM2E2Nm...,110830,JohnBarrett KeithVitali TerryNorton BradMorris...,FransNel,Action,GlobalPictures DistantHorizon CannonGroup,JohnBarrett KeithVitali TerryNorton BradMorris...
4,An American Tail: Fievel Goes West,1991,tt0101329,https://m.media-amazon.com/images/M/MV5BZmUyOT...,10380,PhillipGlasser JamesStewart EricaYohn CathyCav...,SimonWells,Adventure Animation Family Western,AmblinEntertainment UniversalPictures Amblimation,PhillipGlasser JamesStewart EricaYohn CathyCav...


* TfidfVectorizer

In [39]:
movies[movies['title'] == 'Iron Man']

,title,release_year,imdb_id,poster,tmdb_id,cast,director,genres,production_companies,tags
3254,Iron Man,2008,tt0371746,https://m.media-amazon.com/images/M/MV5BMTczNT...,1726,RobertDowneyJr TerrenceHoward JeffBridges Gwyn...,JonFavreau,Action ScienceFiction Adventure,MarvelStudios,RobertDowneyJr TerrenceHoward JeffBridges Gwyn...


In [15]:
tfidf_tags = TfidfVectorizer()   #for tags
tfidf_tags

TfidfVectorizer()

In [16]:
tfidf_tags_matrix = tfidf_tags.fit_transform(movies['tags'])  #tags -> tfidf vector
tfidf_tags_matrix

<6147x24390 sparse matrix of type '<class 'numpy.float64'>'
	with 73040 stored elements in Compressed Sparse Row format>

* cosine similariry

In [17]:
tags_similarity_array = cosine_similarity(tfidf_tags_matrix)
tags_similarity_array

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.72649311,
        0.0135681 ],
       [0.        , 0.        , 0.        , ..., 0.72649311, 1.        ,
        0.01421929],
       [0.        , 0.        , 0.        , ..., 0.0135681 , 0.01421929,
        1.        ]])

In [22]:
#pickle.dump(tags_similarity_array, open('similarity_array.pkl', 'wb'))

* building function

In [53]:
def recommend(t):
    
    if t not in movies['title'].values:
        print("The movie title you entered is not in our database. Please try entering a different title.")
        
    else:
    
        if len(movies.loc[movies['title']==t]) > 1:
        
            #printing release year of movies with same name
            print(movies.loc[movies['title']==t]['release_year'].values)
            print('please provide release year...from above')
        
            r_year = int(input('Enter the release year :'))
        
            if r_year not in movies.loc[movies['title']==t]['release_year'].values:
                print("\n The release year {} you entered is not correct. We have taken the movie that was released first.".format(r_year))
                
                idx =  movies.loc[movies['title']== t].index[0]
                
            else:
                idx =  movies.loc[(movies['title'] == t) & (movies['release_year'] == r_year)].index[0]
    
    
        else:
            idx =  movies.loc[movies['title']== t].index[0]
        
        
        score_lst = list(enumerate(tags_similarity_array[idx]))   #give index and score
        score_lst = sorted(score_lst, key = lambda x:x[1] ,reverse=True)     #sorting descending
        score_lst = score_lst[1:17] #excluding first item since it's the requested movie itself and giving top10
    
        rcmd_lst = []                                             
        for i in range(len(score_lst)):
            a = score_lst[i][0]
            rcmd_lst.append(movies.iloc[a]['title']) 
        
        
        print('\n Here are the recommendations for the movie: {}'.format(t))
        
        return rcmd_lst

In [54]:
recommend('The Avengers')

[1998 2012]
please provide release year...from above

 Here are the recommendations for the movie: The Avengers


['Avengers: Age of Ultron',
 'Avengers: Endgame',
 'Avengers: Infinity War',
 'Captain America: Civil War',
 'Iron Man',
 'Captain America: The Winter Soldier',
 'Ant-Man',
 'Iron Man 2',
 'Guardians of the Galaxy',
 'Serenity',
 'The Incredible Hulk',
 'Zodiac',
 'Eternals',
 'The Perfect Score',
 'The Nanny Diaries',
 'Thor: Ragnarok']

In [12]:
movies.loc[(movies['title'] =='The Avengers') & (movies['release_year'] == 2012) ].index[0]

4035

In [13]:
movies['poster'][0]

'https://m.media-amazon.com/images/M/MV5BMWY2MTY1NWUtN2E1NC00MTg2LWFkOWQtNmE0ODJlMDZhYzUyXkEyXkFqcGdeQXVyNjMwMjk0MTQ@._V1_.jpg'

building engine with different weitages for different features

* genres

In [14]:
#tfidf
genres_tfidf = TfidfVectorizer()
genres_tfidf

TfidfVectorizer()

In [15]:
tfidf_genres_matrix = genres_tfidf.fit_transform(movies['genres'])  #tags -> tfidf vector
tfidf_genres_matrix

<6141x23 sparse matrix of type '<class 'numpy.float64'>'
	with 15445 stored elements in Compressed Sparse Row format>

In [16]:
genres_similarity_array = cosine_similarity(tfidf_genres_matrix)
genres_similarity_array

array([[1.        , 0.49176832, 0.4693157 , ..., 0.        , 0.33940566,
        0.34926628],
       [0.49176832, 1.        , 0.23079459, ..., 0.        , 0.16690895,
        0.17175809],
       [0.4693157 , 0.23079459, 1.        , ..., 0.13670472, 0.58723263,
        0.16391615],
       ...,
       [0.        , 0.        , 0.13670472, ..., 1.        , 0.        ,
        0.57973215],
       [0.33940566, 0.16690895, 0.58723263, ..., 0.        , 1.        ,
        0.11854295],
       [0.34926628, 0.17175809, 0.16391615, ..., 0.57973215, 0.11854295,
        1.        ]])

* other tags

In [17]:
movies['other_tags'] = movies['cast'] + " " + movies['director'] + " " + movies['production_companies']

In [18]:
tfidf_othertags_matrix = tfidf_tags.fit_transform(movies['other_tags'])  #tags -> tfidf vector
tfidf_othertags_matrix

<6141x21686 sparse matrix of type '<class 'numpy.float64'>'
	with 63656 stored elements in Compressed Sparse Row format>

In [19]:
othertags_similarity_array = cosine_similarity(tfidf_othertags_matrix)
othertags_similarity_array

array([[1.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 1.        , 0.03396061, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.03396061, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

* building function

In [20]:
def weighted_recommend(t):
    
    if t not in movies['title'].values:
        print("The movie title you entered is not in our database. Please try entering a different title.")
        
    else:
    
        if len(movies.loc[movies['title']==t]) > 1:
        
            #printing release year of movies with same name
            print(movies.loc[movies['title']==t]['release_year'].values)
            print('please provide release year...from above')
        
            r_year = int(input('Enter the release year :'))
            
            if r_year not in movies.loc[movies['title']==t]['release_year'].values:
                print("Please try to select a valid release year for the movie from above")
                
            else:
                idx =  movies.loc[(movies['title'] == t) & (movies['release_year'] == r_year)].index[0]
    
        else:
            idx =  movies.loc[movies['title']== t].index[0]
        
        
        score_lst = (genres_similarity_array[idx] * 0.4 + othertags_similarity_array[idx] * 0.6)
        score_lst = list(enumerate(score_lst))   #give index and score
        score_lst = sorted(score_lst, key = lambda x:x[1] ,reverse=True)     #sorting descending
        score_lst = score_lst[1:15] #excluding first item since it's the requested movie itself and giving top10
    
        rcmd_lst = []                                             
        for i in range(len(score_lst)):
            a = score_lst[i][0]
            rcmd_lst.append(movies['title'][a]) 
        
        
        print('\n Here are the recommendations for the movie: {}'.format(t))
        
        return rcmd_lst

In [21]:
weighted_recommend('I, Robot')


 Here are the recommendations for the movie: I, Robot


['Battle: Los Angeles',
 'Terminator: Dark Fate',
 'After Earth',
 'Cyborg Cop',
 'Monolith',
 'Cyborg Cop II',
 'Black Scorpion',
 'Barb Wire',
 'Space Marines',
 'Toad Warrior',
 'Double Team',
 'Soldier',
 'The Matrix',
 'Universal Soldier: The Return']

In [22]:
recommend('I, Robot')


 Here are the recommendations for the movie: I, Robot


['Welcome to Mooseport',
 'Bright',
 'Seven Pounds',
 'Hitch',
 'After Earth',
 'The Pursuit of Happyness',
 'Bad Boys for Life',
 'Collateral Beauty',
 'Ali',
 '28 Days',
 'King Richard',
 'Hancock',
 'I Am Legend',
 'This Means War']

In [23]:
def recommend(t):
    
    if t not in movies['title'].values:
        print("The movie title you entered is not in our database. Please try entering a different title.")
        
    else:
    
        if len(movies.loc[movies['title']==t]) > 1:
        
            #printing release year of movies with same name
            print(movies.loc[movies['title']==t]['release_year'].values)
            print('please provide release year...from above')
        
            r_year = int(input('Enter the release year :'))
        
            if r_year not in movies.loc[movies['title']==t]['release_year'].values:
                print("\n The release year {} you entered is not correct. We have taken the movie that was released first.".format(r_year))
                
                idx =  movies.loc[movies['title']== t].index[0]
                
            else:
                idx =  movies.loc[(movies['title'] == t) & (movies['release_year'] == r_year)].index[0]
    
    
        else:
            idx =  movies.loc[movies['title']== t].index[0]
        
        
        score_lst = list(enumerate(tags_similarity_array[idx]))   #give index and score
        score_lst = sorted(score_lst, key = lambda x:x[1] ,reverse=True)     #sorting descending
        score_lst = score_lst[1:15] #excluding first item since it's the requested movie itself and giving top10
    
        rcmd_lst = []                                             
        for i in range(len(score_lst)):
            a = score_lst[i][0]
            rcmd_lst.append(movies['title'][a]) 
        
        
        print('\n Here are the recommendations for the movie: {}'.format(t))
        
        return rcmd_lst

In [24]:
recommend("The Avengers")

[1998 2012]
please provide release year...from above


Enter the release year : 2012



 Here are the recommendations for the movie: The Avengers


['Avengers: Age of Ultron',
 'Avengers: Endgame',
 'Avengers: Infinity War',
 'Captain America: Civil War',
 'Iron Man',
 'Captain America: The Winter Soldier',
 'Ant-Man',
 'Iron Man 2',
 'Guardians of the Galaxy',
 'Serenity',
 'The Incredible Hulk',
 'Zodiac',
 'Eternals',
 'The Perfect Score']